In [1]:
import os
os.environ['KERAS_BACKEND'] = 'jax'
import keras
import numpy as np
from keras.utils import image_dataset_from_directory
from PIL import Image
import io
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow as tf


# Load datasets using Keras utilities
batch_size = 32
img_size = (512, 512)

train_ds = image_dataset_from_directory(
    'official_data/train',
    image_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',
)

val_ds = image_dataset_from_directory(
    'official_data/valid',
    image_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',
)

test_ds = image_dataset_from_directory(
    'official_data/test',
    image_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',
)



# Get class names
class_names = train_ds.class_names
num_classes = len(class_names)
input_shape = (512, 512, 1)

model = keras.Sequential([
    keras.layers.Input(shape=input_shape),
    keras.layers.Rescaling(1./255),  # Normalize pixel values [0,1]
    keras.layers.Normalization(mean=.5, variance=0.25),

    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(32, kernel_size=(3, 3), strides=(2, 2), activation='relu'),
    
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=(2, 2)),
    
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', strides=(2, 2)),
    
    keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dropout(.5),
    keras.layers.Dense(num_classes, activation="softmax"),
])


2025-07-15 09:49:42.722833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752544182.734134  365596 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752544182.738069  365596 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752544182.747567  365596 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752544182.747576  365596 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752544182.747577  365596 computation_placer.cc:177] computation placer alr

Found 20894 files belonging to 7 classes.


2025-07-15 09:49:45.807953: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1752544185.808010  365596 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8670 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:01:00.0, compute capability: 8.6


Found 873 files belonging to 6 classes.
Found 437 files belonging to 6 classes.


In [2]:
# original_model = keras.models.load_model('models/simple_cnn_best_model.keras', compile=False)

In [3]:

# def create_dropout_version(original_model, dropout_rate=0.5):
#     new_model = keras.Sequential()
    
#     for i, layer in enumerate(original_model.layers):
#         new_model.add(layer)
        
#         if isinstance(layer, keras.layers.MaxPooling2D):
#             if i != 10:  
#                 new_model.add(keras.layers.Dropout(dropout_rate, name=f"dropout_{i}"))
        
#         if isinstance(layer, keras.layers.GlobalAveragePooling2D):
#             new_model.add(keras.layers.Dropout(dropout_rate, name="dropout_final"))
    
#     return new_model

# model = create_dropout_version(original_model, dropout_rate=0.15)

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 512, 512, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization (Normalization)   │ (None, 512, 512, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 510, 510, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 254, 254, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 252, 252, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 125, 125, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 123, 123, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 61, 61, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 59, 59, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 583,399 (2.23 MB)

 Trainable params: 583,399 (2.23 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
epochs = 100

# Learning rate scheduler
def cosine_annealing_scheduler(epoch, lr):
    initial_lr = 1e-3
    min_lr = 1e-6
    T_max = int(epochs / 2)
    
    cosine_decay = 0.5 * (1 + np.cos(np.pi * (epoch % T_max) / T_max))
    new_lr = (initial_lr - min_lr) * cosine_decay + min_lr
    
    return float(new_lr)

# Confusion Matrix callback using Keras TensorBoard
class ConfusionMatrixCallback(keras.callbacks.Callback):
    def __init__(self, val_data, class_names=None):
        super().__init__()
        self.val_data = val_data
        self.class_names = class_names
        self.file_writer = tf.summary.create_file_writer('logs/cm')

    def on_epoch_end(self, epoch, logs=None):
        y_true = []
        y_pred = []
        
        for images, labels in self.val_data:
            preds = self.model.predict(images, verbose=0)
            preds = np.argmax(preds, axis=1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds)

        fig = plt.figure(figsize=(15, 15))
        ax = fig.add_subplot(111)
        ConfusionMatrixDisplay.from_predictions(
            y_true,
            y_pred,
            labels=range(len(self.class_names)),
            display_labels=self.class_names,
            ax=ax
        )
        ax.set_title(f"Confusion Matrix Epoch: {epoch}")
        
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        plt.close(fig)
        buf.seek(0)
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        image = tf.expand_dims(image, 0)
        
        with self.file_writer.as_default():
            tf.summary.image("Confusion Matrix", image, step=epoch)


In [6]:
import io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow import keras
 
class ConfusionMatrixCallback(keras.callbacks.Callback):
 
    def __init__(self, val_data, class_names=None):
        super().__init__()
        self.val_data = val_data
        self.class_names = class_names
        self.file_writer = tf.summary.create_file_writer('logs/cm')
 
    def on_epoch_end(self, epoch, logs=None):
        y_true = []
        y_pred = []
        for images, labels in self.val_data:
            preds = self.model.predict(images, verbose=0)
            preds = np.argmax(preds, axis=1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds)
 
        # Numeric confusion matrix
        fig1 = plt.figure(figsize=(10, 10))
        ax1 = fig1.add_subplot(111)
        ConfusionMatrixDisplay.from_predictions(
            y_true,
            y_pred,
            labels=range(len(self.class_names)),
            display_labels=self.class_names,
            ax=ax1
        )
        ax1.set_title(f"Confusion Matrix Epoch: {epoch}")
        buf1 = io.BytesIO()
        plt.savefig(buf1, format='png')
        plt.close(fig1)
        buf1.seek(0)
        image1 = tf.image.decode_png(buf1.getvalue(), channels=4)
        image1 = tf.expand_dims(image1, 0)
 
        # Percentage confusion matrix
        cm = confusion_matrix(y_true, y_pred, labels=range(len(self.class_names)))
        cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
 
        fig2, ax2 = plt.subplots(figsize=(10, 10))
        sns.heatmap(cm_percentage, annot=True, fmt=".1f", cmap="Blues",
                    xticklabels=self.class_names, yticklabels=self.class_names, ax=ax2)
        ax2.set_xlabel('Predicted Label')
        ax2.set_ylabel('True Label')
        ax2.set_title(f"Confusion Matrix (Percentage) Epoch: {epoch}")
        buf2 = io.BytesIO()
        plt.savefig(buf2, format='png')
        plt.close(fig2)
        buf2.seek(0)
        image2 = tf.image.decode_png(buf2.getvalue(), channels=4)
        image2 = tf.expand_dims(image2, 0)
 
        with self.file_writer.as_default():
            tf.summary.image("Confusion Matrix - Count", image1, step=epoch)
            tf.summary.image("Confusion Matrix - Percentage", image2, step=epoch)

In [ ]:
# Callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="models/office_simple_cnn_best_model.keras", save_best_only=True, monitor="val_loss"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=int(epochs/4)),
    keras.callbacks.TensorBoard(log_dir="logs"),
    keras.callbacks.LearningRateScheduler(cosine_annealing_scheduler, verbose=1),
    ConfusionMatrixCallback(val_ds, class_names=class_names),
]

# Compile and fit
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3, weight_decay=1e-6),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks,
)

# Evaluate
score = model.evaluate(test_ds, verbose=0)
print(f"Test loss: {score[0]}")
print(f"Test accuracy: {score[1]}")


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/100


2025-07-15 09:50:04.880930: E external/xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng57{k2=0,k13=2,k14=2,k18=1,k23=0} for conv %cudnn-conv-bw-filter.7 = (f32[32,1,3,3]{3,2,1,0}, u8[0]{0}) custom-call(%multiply.56, %select.22), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", metadata={op_name="jit(train_step)/jit(main)/transpose(jvp(conv2d))/conv_general_dilated" source_file="/home/lesliebinbin/codings/github-dh-cv/.venv/lib/python3.11/site-packages/keras/src/backend/jax/nn.py" source_line=356}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false,"reification_cost":[]} is taking a while...
2025-07-15 09:50:05.072665: E external/xla/xla/service/slow_operation_alarm.cc:140] The operation took 1.192028594s
Trying algorithm eng57{k2=0,k13=2,k14=2,k18=1,k23=0}

532/653 ━━━━━━━━━━━━━━━━━━━━ 14s 121ms/step - acc: 0.3215 - loss: 1.6098